In [63]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from webdriver_manager.firefox import GeckoDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os

In [65]:
#generating URLs for the first five pages
base_url = "https://www.seek.com.au/healthcare-data-analyst-jobs"
page_urls = [base_url]
for p in range(2,6):
    page_urls.append(f"{base_url}?page={p}")
print(page_urls)

#dataframe to store extracted info from jobs 
columns = ['Title', 'Company', 'Location', 'Short Description', 'Link']
job_data = pd.DataFrame(columns=columns)

['https://www.seek.com.au/healthcare-data-analyst-jobs', 'https://www.seek.com.au/healthcare-data-analyst-jobs?page=2', 'https://www.seek.com.au/healthcare-data-analyst-jobs?page=3', 'https://www.seek.com.au/healthcare-data-analyst-jobs?page=4', 'https://www.seek.com.au/healthcare-data-analyst-jobs?page=5']


In [67]:
service = Service(GeckoDriverManager().install())
driver = webdriver.Firefox(service=service)

for url in page_urls:
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    #finding all the job cards
    job_cards = soup.find_all("article", {"data-automation" : "normalJob"})
    
    for card in job_cards:
        title = (card.find("a", {"data-automation" : "jobTitle"})).get_text(strip=True)
        company = (card.find("a", {"data-automation" : "jobCompany"})).get_text(strip=True)
        location = (card.find("a", {"data-automation" : "jobLocation"})).get_text(strip=True)
        short_desc = (card.find("span", {"data-automation" : "jobShortDescription"})).get_text(strip=True)
        link = card.find("a", {"data-automation" : "jobTitle"})["href"]
        
        job_data.loc[len(job_data)] = [title, company, location, short_desc, link]

driver.quit()

In [69]:
# Define your target folder
folder_path = "C:\\Users\\lubab\\OneDrive\\Documents\\Data Science Portfolio Projects\\Healthcare Data Anayst Jobs Australia\\healthcare-data-analyst-jobs-australia\\data\\raw"

# Make sure the folder exists (optional but recommended)
os.makedirs(folder_path, exist_ok=True)

# Full file path
file_path = os.path.join(folder_path, "healthcare_jobs.csv")

# Save CSV
job_data.to_csv(file_path, index=False, encoding="utf-8-sig")

print(f"Data saved to {file_path}")

Data saved to C:\Users\lubab\OneDrive\Documents\Data Science Portfolio Projects\Healthcare Data Anayst Jobs Australia\healthcare-data-analyst-jobs-australia\data\raw\healthcare_jobs.csv
